In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from thermo import Chemical
from thermo.eos import PR

# Global Constants:
R_U = 8.31446 #J/(mol K)
n2o_g = Chemical('nitrous oxide', T=298.15)
PC = n2o_g.Pc
TC = n2o_g.Tc
OMEGA = n2o_g.omega

MW = (n2o_g.MW/1000) #n2o.MW in g/mol --> converted to kg/mol
R_GAS_CONST = R_U/MW
KAPPA = 0.37464 + 1.5422*n2o_g.omega - 0.26992*n2o_g.omega**2
b = 0.07780*(R_U*TC/PC)
g = 9.81 #m/s^2

P_atm = 101325

print("imported libraries")

imported libraries


Check 1: Compare cp ideal gas properties between Chemical and [7] Glenn Polynomial

In [ ]:
def solve_cp_ig_polynomial(T):
    A = 21.62
    B = 72.81
    C = -57.78
    D = 18.3
    E = 0.0
    if 150<T and T<310:
        T_reduced = T/1000

        #NOTE: Thesis has this multiplied by a factor of 1000, but I think that is to convert MW because otherwise magnitude seems 1000x too big
        cp = (A + B*T_reduced + C*T_reduced**2 + D*T_reduced**3 + E/(T_reduced**2) ) / MW

        return cp #J/(kg K)
    raise ValueError("Temperature outside of function bounds!")


temp_arr = np.linspace(270, 305, (305-270) )

cp_chemical_arr = []
cp_polynomial_arr = []


for T in temp_arr:
    n2o_ig_for_cp = Chemical('N2O', T=T, P= P_atm) 
    cp_chemical_arr.append( n2o_ig_for_cp.Cpg )
    cp_polynomial_arr.append( solve_cp_ig_polynomial(T) )


plt.scatter(temp_arr, cp_chemical_arr, label = "chemical library")
plt.scatter(temp_arr, cp_polynomial_arr, label = "ig polynomial")
plt.xlabel('Temp (T)')
plt.ylabel('Cp (J/(kg K))')
plt.title('IDEAL GAS Cp vs. Temperature for Check 1')
plt.legend()
plt.grid(True)
plt.show()

Check 2.1: Compare real gas cp with (ig + PR_ref) to NIST cp
- Where NIST cp taken at P = 1 atm from 270K to 305K (entirely vapor phase)

In [ ]:
P_step_2 = 101325 #Pa

temp_arr = [270,272,274,276,278,280,282,284,286,288,290,292,294,296,298,300,302,304,306]
cp_real_NIST_const_P_arr = [855.6,857.59,859.57,861.55,863.53,865.5,867.46,869.42,871.37,873.32,875.27,877.2,879.13,881.06,882.97,884.88,886.79,888.69,890.58,]
cp_solve_arr = []

for T in temp_arr:
    n2o_ig_step_2= Chemical('N2O', T=T, P= P_atm) 
    preos_g_step_2 = PR(Tc=TC, Pc=PC, omega=OMEGA, T=T, P=P_step_2)
    cp_solve_arr.append( (preos_g_step_2.Cp_dep_g/MW + n2o_ig_step_2.Cpg) )
    
plt.scatter(temp_arr, cp_real_NIST_const_P_arr, label = "NIST", color = 'r')
plt.scatter(temp_arr, cp_solve_arr, label = "ig polynomial + dep", color = 'g')
plt.xlabel('Temp (T)')
plt.ylabel('Cp (J/(kg K))')
plt.title('REAL GAS Cp vs. Temperature for Check 2.1')
plt.legend()
plt.grid(True)
plt.show()


Traceback (most recent call last):
  File "/home/rwright/.vscode-server/extensions/ms-python.python-2025.0.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 19, in <module>
  File "/home/rwright/.local/lib/python3.10/site-packages/matplotlib/pyplot.py", line 527, in show
    return _get_backend_mod().show(*args, **kwargs)
  File "/home/rwright/.local/lib/python3.10/site-packages/matplotlib/backend_bases.py", line 3448, in show
    cls.mainloop()
  File "/home/rwright/.local/lib/python3.10/site-packages/matplotlib/backends/backend_qt.py", line 593, in start_main_loop
    with _maybe_allow_interrupt(qapp):
  File "/usr/lib/python3.10/contextlib.py", line 142, in __exit__
    next(self.gen)
  File "/home/rwright/.local/lib/python3.10/site-packages/matplotlib/backends/qt_compat.py", line 230, in _maybe_allow_interrupt
    old_sigint_handler(*handler_args)
KeyboardInterrupt



Check 2.2: Compare real gas cp with (ig + PR_ref) to NIST cp
- Where NIST cp taken at P = 4 MPa from 270K to 305K (partially liq and vapor)
changes from liq (T= 283.14 K, cp= 2567.3 J/(kg K)) --> vap (T= 283.14 K, cp= 2133.9 J/(kg K)) 


In [ ]:
P_step_2 = 4e6 #Pa

temp_arr = [270,272,274,276,278,280,282,284,286,288,290,292,294,296,298,300,302,304,306]
cp_real_NIST_const_P_arr = [2153.2,2193.2,2238.6,2290.7,2351.3,2423,2509.5,2065.4,1933,1827.8,1742.1,1670.9,1610.8,1559.4,1515,1476.3,1442.2,1412.1,1385.2]

cp_solve_g_arr = []
cp_solve_l_arr = []
T_g_arr = []
T_l_arr = []

T_transition = 283.14


for T in temp_arr:
    n2o_ig_step_2= Chemical('N2O', T=T, P= P_step_2) 
    preos_step_2 = PR(Tc=TC, Pc=PC, omega=OMEGA, T=T, P=P_step_2)
    

    if T > T_transition:
        cp_solve_g_arr.append( (preos_g_step_2.Cp_dep_g/MW + n2o_ig_step_2.Cpg) )

        print("for gas property: dep / ig: ", (preos_g_step_2.Cp_dep_g/MW, n2o_ig_step_2.Cpg))
        T_g_arr.append(T)

    else: 
        cp_solve_l_arr.append( (preos_step_2.Cp_dep_l/MW + n2o_ig_step_2.Cpg) )
        T_l_arr.append(T)
    
plt.scatter(temp_arr, cp_real_NIST_const_P_arr, label = "NIST", color = 'r')
plt.scatter(T_g_arr, cp_solve_g_arr, label = "ig polynomial + dep gas", color = 'orchid')
plt.scatter(T_l_arr, cp_solve_l_arr, label = "ig polynomial + dep liquid", color = 'darkslateblue')

plt.axvline(T_transition)
plt.xlabel('Temp (T)')
plt.ylabel('Cp (J/(kg K))')
plt.title('REAL GAS Cp vs. Temperature for Check 2.2')
plt.legend()
plt.grid(True)
plt.show()

for gas property: dep / ig:  (4.688581361526833, 861.8670484555939)
for gas property: dep / ig:  (4.688581361526833, 863.9991722006504)
for gas property: dep / ig:  (4.688581361526833, 866.1193750750027)
for gas property: dep / ig:  (4.688581361526833, 868.2277450891667)
for gas property: dep / ig:  (4.688581361526833, 870.3243744944992)
for gas property: dep / ig:  (4.688581361526833, 872.4093593309301)
for gas property: dep / ig:  (4.688581361526833, 874.4827989997412)
for gas property: dep / ig:  (4.688581361526833, 876.5447958604858)
for gas property: dep / ig:  (4.688581361526833, 878.5954548511493)
for gas property: dep / ig:  (4.688581361526833, 880.634883130656)
for gas property: dep / ig:  (4.688581361526833, 882.6631897428528)
for gas property: dep / ig:  (4.688581361526833, 884.6804853010901)


Check 3.1: Compare cp ideal gas properties between Chemical and [7] Glenn Polynomial
(note for ideal gas Cp - Cv = R_GAS_Const )

In [ ]:
def solve_cv_ig_polynomial(T):
    A = 21.62
    B = 72.81
    C = -57.78
    D = 18.3
    E = 0.0
    if 150<T and T<310:
        T_reduced = T/1000

        #NOTE: Thesis has this multiplied by a factor of 1000, but I think that is to convert MW because otherwise magnitude seems 1000x too big
        cp = (A + B*T_reduced + C*T_reduced**2 + D*T_reduced**3 + E/(T_reduced**2) ) / MW
        cv = cp - R_GAS_CONST
        return cv #J/(kg K)
    raise ValueError("Temperature outside of function bounds!")


temp_arr = np.linspace(270, 305, (305-270) )

cv_chemical_arr = []
cv_polynomial_arr = []


for T in temp_arr:
    n2o_ig_for_cv = Chemical('N2O', T=T, P= P_atm) 
    cv_chemical_arr.append( n2o_ig_for_cv.Cvg )
    cv_polynomial_arr.append( solve_cv_ig_polynomial(T) )


plt.scatter(temp_arr, cv_chemical_arr, label = "chemical library")
plt.scatter(temp_arr, cv_polynomial_arr, label = "ig polynomial")
plt.xlabel('Temp (T)')
plt.ylabel('Cv (J/(kg K))')
plt.title('IDEAL GAS Cv vs. Temperature for Check 3')
plt.legend()
plt.grid(True)
plt.show()

Traceback (most recent call last):
  File "/home/rwright/.vscode-server/extensions/ms-python.python-2025.0.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 29, in <module>
  File "/home/rwright/.local/lib/python3.10/site-packages/matplotlib/pyplot.py", line 3684, in scatter
    __ret = gca().scatter(
  File "/home/rwright/.local/lib/python3.10/site-packages/matplotlib/__init__.py", line 1465, in inner
    return func(ax, *map(sanitize_sequence, args), **kwargs)
  File "/home/rwright/.local/lib/python3.10/site-packages/matplotlib/axes/_axes.py", line 4652, in scatter
    raise ValueError("x and y must be the same size")
ValueError: x and y must be the same size



Check 3.2: Compare real gas cv with (ig + PR_ref) to NIST cv
- Where NIST cv taken at P = 4 MPa from 270K to 305K (partially liq and vapor)
changes from liq (T= 283.14 K, cv= 916.95 J/(kg K)) --> vap (T= 283.14 K, cv= 923.2 J/(kg K)) 

In [ ]:
P_step_2 = 4e6 #Pa

temp_arr = [270,272,274,276,278,280,282,284,286,288,290,292,294,296,298,300,302,304,306]
cv_real_NIST_const_P_arr = [909.36,909.82,910.45,911.3,912.4,913.83,915.67,914.02,895.29,879.49,866.05,854.53,844.6,836.01,828.56,822.08,816.42,811.49,807.19,]

cv_solve_g_arr = []
cv_solve_l_arr = []
T_g_arr = []
T_l_arr = []

T_transition = 283.14


for T in temp_arr:
    n2o_ig_step_2= Chemical('N2O', T=T, P= P_step_2) 
    preos_step_2 = PR(Tc=TC, Pc=PC, omega=OMEGA, T=T, P=P_step_2)
    

    if T > T_transition:
        cv_solve_g_arr.append( (preos_g_step_2.Cv_dep_g/MW + n2o_ig_step_2.Cvg) )

        #print("for gas property: dep / ig: ", (preos_g_step_2.Cv_dep_g/MW, n2o_ig_step_2.Cvg))
        T_g_arr.append(T)

    else: 
        cv_solve_l_arr.append( (preos_step_2.Cv_dep_l/MW + n2o_ig_step_2.Cvg) )
        T_l_arr.append(T)
    
plt.scatter(temp_arr, cv_real_NIST_const_P_arr, label = "NIST", color = 'r')
plt.scatter(T_g_arr, cv_solve_g_arr, label = "ig polynomial + dep gas", color = 'orchid')
plt.scatter(T_l_arr, cv_solve_l_arr, label = "ig polynomial + dep liquid", color = 'darkslateblue')

plt.axvline(T_transition)
plt.xlabel('Temp (T)')
plt.ylabel('Cv (J/(kg K))')
plt.title('REAL GAS Cv vs. Temperature for Check 2.2')
plt.legend()
plt.grid(True)
plt.show()

Check 4.1: Want to compare Chemical ideal gas enthalpy --> need to integrate cp polynomial to solve ideal gas enthalpy!

NOTE: CHEMICAL LIB USES T = 298.15 K, P = 101325 Pa AS REF STATE!

In [ ]:
def solve_cp_ig_polynomial(T):
    # Polynomial coefficients
    A = 21.62
    B = 72.81
    C = -57.78  
    D = 18.3
    E = 0.0

    # Apply temperature limits
    if 150 < T and T < 310:
        T_reduced = T / 1000
        cp = (A + B * T_reduced + C * T_reduced**2 + D * T_reduced**3 + E / (T_reduced**2)) / MW  # J/(kg K)
        return cp
    raise ValueError("Temperature outside of function bounds!")

def integrate_polynomial_ig_enthalpy(T_REF, T):
    N = 1000  # Number of intervals for numerical integration
    dt = (T - T_REF) / N
    integral = 0
    for i in range(N):
        Ti = T_REF + i * dt
        Ti_next = Ti + dt
        # Midpoint for integration
        integral += solve_cp_ig_polynomial((Ti + Ti_next) / 2) * dt
    return integral


T_REF = 298.15  # Reference temperature (K)



temp_arr = [270,272,274,276,278,280,282,284,286,288,290,292,294,296,298,300,302,304,306]
polynomial_trapezoid_ig_enthalpy =[]
chemical_lib_ig_enthalpy = []

for T in temp_arr:
    n2o_ig_check_4 = Chemical('N2O', T=T, P= P_atm) 
    chemical_lib_ig_enthalpy.append( n2o_ig_check_4.H )
    polynomial_trapezoid_ig_enthalpy.append( integrate_polynomial_ig_enthalpy(T_REF, T) )


plt.scatter(temp_arr, chemical_lib_ig_enthalpy, label = "chemical library")
plt.scatter(temp_arr, polynomial_trapezoid_ig_enthalpy, label = "ig polynomial")
plt.xlabel('Temp (T)')
plt.ylabel('h (J/kg)')
plt.title('IDEAL GAS Enthalpy vs. Temperature for Check 4')
plt.legend()
plt.grid(True)
plt.show()




Enthalpy at 170 K: -103.87 J/kg


Check 4.2: Want to compare REAL FLUID ENTHALPY, so using ideal gas enthalpy + PR EOS departure enthalpy and comparing to NIST VALUES!!!
- Where NIST cv taken at P = 4 MPa from 270K to 305K (partially liq and vapor)
changes from liq (T= 283.14 K, h= 188.57 kJ/kg) --> vap (T= 283.14 K, h= 393.50 kJ/kg) 

NOTE: NIST ENTHALPY CONVENTION IS THE NORMAL BOILING POINT:

T_boiling = 182.5K for nitrous oxide

In [ ]:
P_step_4 = 4e6 #Pa

NIST_enthalpy_arr = [158030.0, 162380.0, 166810.0, 171340.0, 175980.0, 180750.0, 185680.0, 395310.0, 399300.0, 403060.0, 406630.0, 410040.0, 413320.0, 416490.0, 419560.0, 422550.0, 425470.0, 428320.0, 431120.0]

temp_arr = [270,272,274,276,278,280,282,284,286,288,290,292,294,296,298,300,302,304,306]

ig_dep_enthalpy_l_arr =[]
ig_dep_enthalpy_g_arr =[]

T_g_arr = []
T_l_arr = []

T_BOIL = 182.5

T_transition = 283.14
n2o_ig_ref= Chemical('N2O', T=T_BOIL, P= P_atm) 
n2o_ref_convers = n2o_ig_ref.H


for T in temp_arr:
    n2o_ig_step_4= Chemical('N2O', T=T, P= P_step_4) 
    preos_step_4 = PR(Tc=TC, Pc=PC, omega=OMEGA, T=T, P=P_step_4)
    

    if T > T_transition:
        ig_dep_enthalpy_g_arr.append( (preos_step_4.H_dep_g/MW + n2o_ig_step_4.H - n2o_ref_convers) )
        T_g_arr.append(T)

    else: 
        #ig_dep_enthalpy_l_arr.append( (preos_step_4.h_dep_l/MW + n2o_ig_step_4.H) )
        ig_dep_enthalpy_l_arr.append( (preos_step_4.H_dep_l/MW + n2o_ig_step_4.H - n2o_ref_convers) )
        T_l_arr.append(T)
    
plt.scatter(temp_arr, NIST_enthalpy_arr, label = "NIST", color = 'r')
plt.scatter(T_g_arr, ig_dep_enthalpy_g_arr, label = "ig polynomial + dep gas", color = 'orchid')
plt.scatter(T_l_arr, ig_dep_enthalpy_l_arr, label = "ig polynomial + dep liquid", color = 'darkslateblue')

plt.axvline(T_transition)
plt.xlabel('Temp (T)')
plt.ylabel('h (J/kg)')
plt.title('REAL GAS h vs. Temperature for Check 2.2')
plt.legend()
plt.grid(True)
plt.show()

18 19
Traceback (most recent call last):
  File "/home/rwright/.vscode-server/extensions/ms-python.python-2025.0.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 31, in <module>
  File "/home/rwright/.local/lib/python3.10/site-packages/matplotlib/pyplot.py", line 3684, in scatter
    __ret = gca().scatter(
  File "/home/rwright/.local/lib/python3.10/site-packages/matplotlib/__init__.py", line 1465, in inner
    return func(ax, *map(sanitize_sequence, args), **kwargs)
  File "/home/rwright/.local/lib/python3.10/site-packages/matplotlib/axes/_axes.py", line 4652, in scatter
    raise ValueError("x and y must be the same size")
ValueError: x and y must be the same size



Check 5.1: Want to compare Chemical ideal gas internal energy --> need to integrate cp polynomial to solve ideal gas internal energy!

NOTE: CHEMICAL LIB USES T = 298.15 K, P = 101325 Pa AS REF STATE!

In [ ]:
def solve_cp_ig_polynomial(T):
    # Polynomial coefficients
    A = 21.62
    B = 72.81
    C = -57.78  
    D = 18.3
    E = 0.0

    # Apply temperature limits
    if 150 < T and T < 310:
        T_reduced = T / 1000
        cp = (A + B * T_reduced + C * T_reduced**2 + D * T_reduced**3 + E / (T_reduced**2)) / MW  # J/(kg K)
        return cp
    raise ValueError("Temperature outside of function bounds!")
# Reference temperature (e.g., 298.15 K) and reference internal energy (U_ref) in J/mol
T_ref = 298.15
U_ref = 0  # Ideal gas internal energy at T_ref (usually taken as 0 at 298.15 K)

# Function to calculate internal energy at any temperature T using trapezoidal rule
def U_trapezoidal(T, T_ref, U_ref, n_steps=1000):
    # Create an array of temperatures between T_ref and T
    T_vals = np.linspace(T_ref, T, n_steps)
    
    # Calculate the trapezoidal sum for the integral of cp(T)
    integral = 0
    for i in range(1, len(T_vals)):
        # Trapezoidal rule: (cp(T[i-1]) + cp(T[i])) / 2 * (T[i] - T[i-1])
        integral += 0.5 * (solve_cp_ig_polynomial(T_vals[i-1]) + solve_cp_ig_polynomial(T_vals[i])) * (T_vals[i] - T_vals[i-1])
    
    return (U_ref + integral)


U_at_T_trapezoidal = U_trapezoidal(T_ref, T_ref, U_ref)
print(f"Internal Energy at {T_ref} K using trapezoidal method: {U_at_T_trapezoidal} kJ/kg")

n2o_ig_ref_check = Chemical('N2O', T=T_ref, P= P_atm) 
U_at_T_chem_lib = n2o_ig_ref_check.U + 56323.5474589753
print("now with chemical: ",  U_at_T_chem_lib )


temp_arr = [270,272,274,276,278,280,282,284,286,288,290,292,294,296,298,300,302,304,306]
polynomial_trapezoid_ig_int_energy =[]
chemical_lib_ig_int_energy = []

for T in temp_arr:
    n2o_ig_check_4 = Chemical('N2O', T=T, P= P_atm) 
    chemical_lib_ig_int_energy.append( n2o_ig_check_4.U + 56323.5474589753 )
    polynomial_trapezoid_ig_int_energy.append( integrate_polynomial_ig_enthalpy(T_REF, T) )


plt.scatter(temp_arr, chemical_lib_ig_int_energy, label = "chemical library")
plt.scatter(temp_arr, polynomial_trapezoid_ig_int_energy, label = "ig polynomial")
plt.xlabel('Temp (T)')
plt.ylabel('u (J/kg)')
plt.title('IDEAL GAS Internal Energy vs. Temperature for Check 5.1')
plt.legend()
plt.grid(True)
plt.show()


Internal Energy at 298.15 K using trapezoidal method: 0.0 kJ/kg
now with chemical:  0.0
